# Amazon SageMaker で EfficientDet (PyTorch) を転移学習して AWS Lambda にデプロイして推論する

このノートブックは、物体検知のための機械学習アルゴリズム [EfficientDet (PyTorch 実装)](https://github.com/zylo117/Yet-Another-EfficientDet-Pytorch) を Amazon SageMaker の学習ジョブを使って転移学習し、学習したモデルを AWS Lambda にデプロイして推論を実行するサンプルノートブックです。Amazon SageMaker ノートブックインスタンス上でこのノートブックを使用してください。ノートブックインスタンスの起動方法は [こちらの記事](https://qiita.com/mariohcat/items/304b5e9ba20c7761084f) をご参照ください。ノートブックインスタンスタイプはデフォルトの ml.t2.medium で構いませんが、コンテナのビルドに 10分ほどかかるため、時間を短縮したい場合は ml.c5.xlarge などのインスタンスを選択してください。

### 注意
- **このノートブックでは GPU インスタンスを使ってモデルの学習をします。学習時間が数時間になる場合もありますので、[料金](https://aws.amazon.com/jp/sagemaker/pricing/) にご注意ください。**
- このノートブックは、Amazon SageMaker Ground Truth を使ってあらかじめ 100枚程度の画像に対して物体検知（境界ボックス）のラベリングを実施してある想定です。Amazon SageMaker Ground Truth の使い方は [こちらの記事](https://aws.amazon.com/jp/builders-flash/202003/sagemaker-groundtruth-cat/?awsf.filter-name=*all) をご参照ください。

このノートブックの全体の流れは、

1. 学習に使用するデータの準備
1. SageMaker 学習ジョブを使ってモデルの学習
1. ノートブックインスタンス上で推論（モデルがうまく学習できているかの確認）
1. AWS Lambda で推論

## 準備

### ロールにポリシーを追加

**このサンプルでは、コンテナを Amazon ECR に push および AWS Lambda 関数の実行を行います。**以下の操作でこのノートブックインスタンスで使用している IAM ロールに Amazon ECR にイメージを push するための権限と AWS Lambda 関数を実行するための権限を追加してください。

1. Amazon SageMaker コンソールからこのノートブックインスタンスの詳細画面を表示<br>
（左側のメニューのインスタンス -> ノートブックインスタンス -> インスタンス名をクリック）
1. 「アクセス許可と暗号化」の「IAM ロール ARN」のリンクをクリック（IAM のコンソールに遷移します）
1. 「ポリシーをアタッチします」と書いてある青いボタンをクリック
1. 検索ボックスに ec2containerregistry と入力し AmazonEC2ContainerRegistryFullAccess のチェックボックスをチェックする
1. 「ポリシーのアタッチ」と書いてある青いボタンをクリック
1. 「インラインポリシーの追加」をクリック
1. 「サービス」で Lambda を選択
1. 「アクション」で「InvokeFunction」を検索して選択
1. 「リソース」で「ARN の追加」と書かれたリンクをクリックして「Region」に使用しているリージョン名、「Function name」に呼び出したい Lambda 関数名を入力して「追加」ボタンをクリック
1. 「ポリシーの確認」ボタンをクリック
1. 「名前」に任意の名前を入力して「ポリシーの作成」ボタンをクリック

### EfficientDet のソースコードのセットアップ

以下のセルを実行して、今回使用する EfficientDet のリポジトリをクローンします。

In [ ]:
!git clone https://github.com/zylo117/Yet-Another-EfficientDet-Pytorch

以下のセルを実行して、クローンしたソースコードから必要なものを src フォルダにコピーします。src の中のソースコードは、モデルを学習する際に使用します。

In [ ]:
!mkdir src
!mkdir -p docker/train
!cp Yet-Another-EfficientDet-Pytorch/backbone.py src
!cp -r Yet-Another-EfficientDet-Pytorch/efficientdet src
!cp -r Yet-Another-EfficientDet-Pytorch/efficientnet src
!cp -r Yet-Another-EfficientDet-Pytorch/utils src
!pip install pycocotools tqdm pyyaml webcolors

### SageMaker を使うための準備

以下のセルでは、Amazon SageMaker を使うためのセットアップを行います。ロールの情報、ノートブックインスタンスのリージョン、アカウントID などの情報を取得しています。

In [ ]:
%matplotlib inline

import boto3
import sys
import sagemaker
import numpy as np
from sagemaker import get_execution_role

role = get_execution_role()
region = boto3.session.Session().region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')
session = sagemaker.Session()
bucket = session.default_bucket()
s3_prefix = 'EfficientDet-PyTorch'

### アノテーションデータを COCO形式に変換

ここからは、アノテーションデータを COCO形式に変換します。今回使用する EfficientDet の実装は COCO形式のアノテーションファイルを前提としているため、COCO形式以外のアノテーションデータしかない場合は、COCO形式への変換が必要です。

このサンプルでは、Amazon SageMaker Ground Truth を使ってアノテーションした結果のファイルである manifest ファイルを COCO 形式に変換します。<b>このサンプルでは物体検知モデルを学習するため、SageMaker Ground Truth でラベリングする際に「境界ボックス」のラベリングタイプを指定してラベリングしてあることを前提としています。</b>

以下のセルに Amazon SageMaker Ground Truth の output.manifest ファイルが保存されている S3 パスを入力して実行してください。

In [ ]:
manifest_s3_path = 's3://gt_output_path/manifests/output/output.manifest'
label_s3_path = 's3://gt_output_path/annotation-tool/data.json'

In [ ]:
!aws s3 cp $manifest_s3_path ./output.manifest
!aws s3 cp $label_s3_path ./data.json

以下のセルを実行して、output.manifest を COCO形式に変換して、さらに学習用と検証用に分割します。

In [ ]:
# ラベリングしたデータを学習用と検証用に分ける際の学習用データの割合
train_data_ratio = 0.8

In [ ]:
import glob
import json
import ntpath
import os
import shutil
import sys
import re
import random
import boto3

# 学習データ出力フォルダ名
coco_output_dir = 'datasets-gt-cats'

s3 = boto3.resource('s3') #S3オブジェクトを取得

from argparse import Namespace, ArgumentParser


def find_category_name(dic, class_id):

    for k, v in dic.items():
        if int(k)== class_id:
            return v
        
    return 'none'

def load_annotation_bbox(line):
    info = Namespace()
    metadata = re.search(r'"([\w-]+-metadata)"', line).group()[1:-1]
    json_load = json.loads(line)
    job_name = metadata[:-9]

    # Image information
    info.image_fn = json_load['source-ref']
    info.width = json_load[job_name]['image_size'][0]['width']
    info.height =  json_load[job_name]['image_size'][0]['height']
    
    # Bounding boxes
    objects = []
    for obj in json_load[job_name]['annotations']:
        label = find_category_name(json_load[metadata]['class-map'], obj['class_id'])
        class_id = obj['class_id']
        xmin = obj['left']
        ymin = obj['top']
        width = obj['width']
        height = obj['height']

        objects.append(Namespace(label=label, class_id=class_id, xmin=xmin, ymin=ymin, width=width, height=height))
    info.objects = objects

    return info

root_dir = './'

label_exists = False

coco_file_name_train =os.path.join(root_dir, coco_output_dir, 'annotations/instances_train.json')
coco_file_name_validation =os.path.join(root_dir, coco_output_dir, 'annotations/instances_validation.json')


if not os.path.exists(os.path.join(root_dir, coco_output_dir, 'annotations')):
    os.makedirs(os.path.join(root_dir, coco_output_dir, 'annotations'))
    os.makedirs(os.path.join(root_dir, coco_output_dir, 'annotations/train'))
    os.makedirs(os.path.join(root_dir, coco_output_dir, 'annotations/validation'))

images_train = []
images_validation = []
annotations_train = []
annotations_validation = []
image_suffix = ["jpg", "png", "jpeg"]

with open('output.manifest', 'r') as f:
    line = f.readline()

    while line:
        line = line.strip()
        info = load_annotation_bbox(line)
        
        s3path = json.loads(line)['source-ref']
        bucket_name = s3path.split('/')[2]

        bucket_boto3 = s3.Bucket(bucket_name)
        path = s3path[6+len(bucket_name):]
#         print(bucket_name, path)
        
        rd = random.random()
        
        dirname = 'train'
        
        if rd < train_data_ratio:
            images = images_train
            annotations = annotations_train
        else:
            images = images_validation
            annotations = annotations_validation
            dirname = 'validation'
    
        image_id = len(images)
        image_base_fn = os.path.basename(info.image_fn)
        bucket_boto3.download_file(path, os.path.join(root_dir, coco_output_dir, 'annotations', dirname, image_base_fn))
        
        images.append({
            "file_name": image_base_fn,
            "height": info.height,
            "width": info.width,
            "id": image_id
        })
        
        # Annotation information
        for obj in info.objects:

            annotations.append({
                "image_id": image_id,
                "bbox": [obj.xmin, obj.ymin, obj.width, obj.height],
                "category_id": obj.class_id  + 1, # Category ID is zero indexed!!
                "id": len(annotations),
                "iscrowd": 0,
                "area": (obj.width) * (obj.height)
            })

        line = f.readline()

labels = []
with open('data.json', 'r') as f:
    line = f.read()
    line_json = json.loads(line)
    labels_list = line_json['labels']
    
    for l in labels_list:
        labels.append(l['label'])
        
# Categories
categories = []
for i, c in enumerate(labels):
    categories.append({'supercategory': c, 'name': c, 'id': i+1})


# Save annotation file
json.dump(
    {"images": images_train, "annotations": annotations_train, "categories": categories},
    open(coco_file_name_train, "w")
)
json.dump(
    {"images": images_validation, "annotations": annotations_validation, "categories": categories},
    open(coco_file_name_validation, "w")
)
        

### モデルの学習に必要なデータを Amazon S3 にアップロード

学習データ、検証データ、アノテーションデータを 、あとで学習ジョブで使用できるように Amazon S3 にアップロードします。

以下のセルに、アノテーションデータをアップロードしたい S3 パスを入力して実行してください。

In [ ]:
train_data_s3_path = 's3://bucket/path'

In [ ]:
!aws s3 cp $coco_output_dir/annotations $train_data_s3_path/annotations --recursive

## モデルの学習

ここからは、EfficientDet のモデルを SageMaker の学習ジョブの機能を使って学習するための手順です。まずは学習を行うスクリプト `train.py` を作成します。このスクリプトは、初めにクローンしたリポジトリのコードをベースに、SageMaker の学習ジョブ用にカスタマイズしたものです。このサンプルノートブックでは、以下のセルを実行することで src/train.py が作成されますが、別途作成した train.py を src フォルダにコピーしても問題ありません。

In [ ]:
%%writefile src/train.py

# original author: signatrix
# adapted from https://github.com/signatrix/efficientdet/blob/master/train.py
# modified by Zylo117

import argparse
import datetime
import os
import traceback
import json
import logging
import sys

import numpy as np
import torch
import yaml
from tensorboardX import SummaryWriter
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm.autonotebook import tqdm

from backbone import EfficientDetBackbone
from efficientdet.dataset import CocoDataset, Resizer, Normalizer, Augmenter, collater
from efficientdet.loss import FocalLoss
from utils.sync_batchnorm import patch_replication_callback
from utils.utils import replace_w_sync_bn, CustomDataParallel, get_last_weights, init_weights, boolean_string

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

class Params:
    def __init__(self, project_file):
        self.params = yaml.safe_load(open(project_file).read())

    def __getattr__(self, item):
        return self.params.get(item, None)


def get_args():
    parser = argparse.ArgumentParser('Yet Another EfficientDet Pytorch: SOTA object detection network - Zylo117')
    parser.add_argument('-c', '--compound_coef', type=int, default=0, help='coefficients of efficientdet')
    parser.add_argument('-n', '--num_workers', type=int, default=12, help='num_workers of dataloader')
    parser.add_argument('--batch_size', type=int, default=12, help='The number of images per batch among all devices')
    parser.add_argument('--head_only', type=boolean_string, default=False,
                        help='whether finetunes only the regressor and the classifier, '
                             'useful in early stage convergence or small/easy dataset')
    parser.add_argument('--lr', type=float, default=1e-4)
    parser.add_argument('--optim', type=str, default='adamw', help='select optimizer for training, '
                                                                   'suggest using \'admaw\' until the'
                                                                   ' very final stage then switch to \'sgd\'')
    parser.add_argument('--num_epochs', type=int, default=500)
    parser.add_argument('--val_interval', type=int, default=1, help='Number of epoches between valing phases')
    parser.add_argument('--save_interval', type=int, default=500, help='Number of steps between saving')
    parser.add_argument('--es_min_delta', type=float, default=0.0,
                        help='Early stopping\'s parameter: minimum change loss to qualify as an improvement')
    parser.add_argument('--es_patience', type=int, default=0,
                        help='Early stopping\'s parameter: number of epochs with no improvement after which training will be stopped. Set to 0 to disable this technique.')
    parser.add_argument('--data_path', type=str, default='datasets/', help='the root folder of dataset')
    parser.add_argument('--log_path', type=str, default='logs/')
    parser.add_argument('-w', '--load_weights', type=str, default=None,
                        help='whether to load weights from a checkpoint, set None to initialize, set \'last\' to load last checkpoint')
    parser.add_argument('--saved_path', type=str, default='logs/')
    parser.add_argument('--debug', type=boolean_string, default=False,
                        help='whether visualize the predicted boxes of training, '
                             'the output images will be in test/')
    parser.add_argument('--init', type=boolean_string, default=False,
                        help='whether start training from epoch 0')
    
    parser.add_argument('--hosts', type=list, default=json.loads(os.environ['SM_HOSTS']))
    parser.add_argument('--current-host', type=str, default=os.environ['SM_CURRENT_HOST'])
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--train-dir', type=str, default=os.environ['SM_CHANNEL_TRAIN'])
    parser.add_argument('--validation-dir', type=str, default=os.environ['SM_CHANNEL_VALIDATION'])
    parser.add_argument('--annotation-dir', type=str, default=os.environ['SM_CHANNEL_ANNOTATIONS'])
    parser.add_argument('--num-gpus', type=int, default=os.environ['SM_NUM_GPUS'])
    parser.add_argument("--checkpoint-path",type=str,default="/opt/ml/checkpoints")
    parser.add_argument('--output-dir', type=str, default=os.environ['SM_OUTPUT_DIR'])
    
    args = parser.parse_args()
    return args


class ModelWithLoss(nn.Module):
    def __init__(self, model, debug=False):
        super().__init__()
        self.criterion = FocalLoss()
        self.model = model
        self.debug = debug

    def forward(self, imgs, annotations, obj_list=None):
        _, regression, classification, anchors = self.model(imgs)
        if self.debug:
            cls_loss, reg_loss = self.criterion(classification, regression, anchors, annotations,
                                                imgs=imgs, obj_list=obj_list)
        else:
            cls_loss, reg_loss = self.criterion(classification, regression, anchors, annotations)
        return cls_loss, reg_loss


def train(opt):
    params = Params(f'settings.yml')
    

    if opt.num_gpus == 0:
        os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

    if torch.cuda.is_available():
        torch.cuda.manual_seed(42)
    else:
        torch.manual_seed(42)

    opt.saved_path = opt.checkpoint_path + f'/'
    opt.log_path = opt.output_dir + f'/tensorboard/'
    os.makedirs(opt.log_path, exist_ok=True)
    os.makedirs(opt.saved_path, exist_ok=True)

    training_params = {'batch_size': opt.batch_size,
                       'shuffle': True,
                       'drop_last': True,
                       'collate_fn': collater,
                       'num_workers': opt.num_workers}

    val_params = {'batch_size': opt.batch_size,
                  'shuffle': False,
                  'drop_last': True,
                  'collate_fn': collater,
                  'num_workers': opt.num_workers}

    input_sizes = [512, 640, 768, 896, 1024, 1280, 1280, 1536, 1536]
    training_set = CocoDataset(root_dir='/opt/ml/input/data', set=params.train_set,
                               transform=transforms.Compose([Normalizer(mean=params.mean, std=params.std),
                                                             Augmenter(),
                                                             Resizer(input_sizes[opt.compound_coef])]))

    training_generator = DataLoader(training_set, **training_params)

    val_set = CocoDataset(root_dir='/opt/ml/input/data', set=params.val_set,
                          transform=transforms.Compose([Normalizer(mean=params.mean, std=params.std),
                                                        Resizer(input_sizes[opt.compound_coef])]))
    val_generator = DataLoader(val_set, **val_params)

    model = EfficientDetBackbone(num_classes=len(params.obj_list), compound_coef=opt.compound_coef,
                                 ratios=eval(params.anchors_ratios), scales=eval(params.anchors_scales))

    # load last weights
    if opt.load_weights is not None:
        if opt.load_weights.endswith('.pth'):
            weights_path = opt.load_weights
        else:
            weights_path = get_last_weights(opt.saved_path)
        try:
            last_step = int(os.path.basename(weights_path).split('_')[-1].split('.')[0])
        except:
            last_step = 0

        try:
            ret = model.load_state_dict(torch.load(weights_path), strict=False)
        except RuntimeError as e:
            print(f'[Warning] Ignoring {e}')
            print(
                '[Warning] Don\'t panic if you see this, this might be because you load a pretrained weights with different number of classes. The rest of the weights should be loaded already.')

        print(f'[Info] loaded weights: {os.path.basename(weights_path)}, resuming checkpoint from step: {last_step}')
    else:
        last_step = 0
        print('[Info] initializing weights...')
        init_weights(model)
        
    if opt.init:
        last_step = 0
        

    # freeze backbone if train head_only
    if opt.head_only:
        def freeze_backbone(m):
            classname = m.__class__.__name__
            for ntl in ['EfficientNet', 'BiFPN']:
                if ntl in classname:
                    for param in m.parameters():
                        param.requires_grad = False

        model.apply(freeze_backbone)
        print('[Info] freezed backbone')

    # https://github.com/vacancy/Synchronized-BatchNorm-PyTorch
    # apply sync_bn when using multiple gpu and batch_size per gpu is lower than 4
    #  useful when gpu memory is limited.
    # because when bn is disable, the training will be very unstable or slow to converge,
    # apply sync_bn can solve it,
    # by packing all mini-batch across all gpus as one batch and normalize, then send it back to all gpus.
    # but it would also slow down the training by a little bit.
    if opt.num_gpus > 1 and opt.batch_size // opt.num_gpus < 4:
        model.apply(replace_w_sync_bn)
        use_sync_bn = True
    else:
        use_sync_bn = False

    writer = SummaryWriter(opt.log_path + f'/{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}/')

    # warp the model with loss function, to reduce the memory usage on gpu0 and speedup
    model = ModelWithLoss(model, debug=opt.debug)

    if opt.num_gpus > 0:
        model = model.cuda()
        if opt.num_gpus > 1:
            model = CustomDataParallel(model, opt.num_gpus)
            if use_sync_bn:
                patch_replication_callback(model)

    if opt.optim == 'adamw':
        optimizer = torch.optim.AdamW(model.parameters(), opt.lr)
    else:
        optimizer = torch.optim.SGD(model.parameters(), opt.lr, momentum=0.9, nesterov=True)

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True)

    epoch = 0
    best_loss = 1e5
    best_epoch = 0
    step = max(0, last_step)
    model.train()

    num_iter_per_epoch = len(training_generator)

    try:
        for epoch in range(opt.num_epochs):
            last_epoch = step // num_iter_per_epoch
            if epoch < last_epoch:
                continue

            epoch_loss = []
            progress_bar = tqdm(training_generator)
            for iter, data in enumerate(progress_bar):
                if iter < step - last_epoch * num_iter_per_epoch:
                    progress_bar.update()
                    continue
                try:
                    imgs = data['img']
                    annot = data['annot']

                    if opt.num_gpus == 1:
                        # if only one gpu, just send it to cuda:0
                        # elif multiple gpus, send it to multiple gpus in CustomDataParallel, not here
                        imgs = imgs.cuda()
                        annot = annot.cuda()

                    optimizer.zero_grad()
                    cls_loss, reg_loss = model(imgs, annot, obj_list=params.obj_list)
                    cls_loss = cls_loss.mean()
                    reg_loss = reg_loss.mean()

                    loss = cls_loss + reg_loss
                    if loss == 0 or not torch.isfinite(loss):
                        continue

                    loss.backward()
                    # torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
                    optimizer.step()

                    epoch_loss.append(float(loss))

                    logger.info(
                        'Step: {}. Epoch: {}/{}. Iteration: {}/{}. Train Cls loss: {:.5f}, Train Reg loss: {:.5f}, Train Total loss: {:.5f}'.format(
                            step, epoch, opt.num_epochs, iter + 1, num_iter_per_epoch, cls_loss.item(),
                            reg_loss.item(), loss.item()))
                    writer.add_scalars('Loss', {'train': loss}, step)
                    writer.add_scalars('Regression_loss', {'train': reg_loss}, step)
                    writer.add_scalars('Classfication_loss', {'train': cls_loss}, step)

                    # log learning_rate
                    current_lr = optimizer.param_groups[0]['lr']
                    writer.add_scalar('learning_rate', current_lr, step)

                    step += 1

                    if step % opt.save_interval == 0 and step > 0:
                        save_checkpoint(model, f'efficientdet-d{opt.compound_coef}_{epoch}_{step}.pth')
                        print('checkpoint...')

                except Exception as e:
                    print('[Error]', traceback.format_exc())
                    print(e)
                    continue
            scheduler.step(np.mean(epoch_loss))

            if epoch % opt.val_interval == 0:
                model.eval()
                loss_regression_ls = []
                loss_classification_ls = []
                for iter, data in enumerate(val_generator):
                    with torch.no_grad():
                        imgs = data['img']
                        annot = data['annot']

                        if opt.num_gpus == 1:
                            imgs = imgs.cuda()
                            annot = annot.cuda()

                        cls_loss, reg_loss = model(imgs, annot, obj_list=params.obj_list)
                        cls_loss = cls_loss.mean()
                        reg_loss = reg_loss.mean()

                        loss = cls_loss + reg_loss
                        if loss == 0 or not torch.isfinite(loss):
                            continue

                        loss_classification_ls.append(cls_loss.item())
                        loss_regression_ls.append(reg_loss.item())

                cls_loss = np.mean(loss_classification_ls)
                reg_loss = np.mean(loss_regression_ls)
                loss = cls_loss + reg_loss

                logger.info(
                    'Val. Epoch: {}/{}. Classification loss: {:1.5f}, Regression loss: {:1.5f}, Val Total loss: {:1.5f}'.format(
                        epoch, opt.num_epochs, cls_loss, reg_loss, loss))
                writer.add_scalars('Loss', {'val': loss}, step)
                writer.add_scalars('Regression_loss', {'val': reg_loss}, step)
                writer.add_scalars('Classfication_loss', {'val': cls_loss}, step)

                if loss + opt.es_min_delta < best_loss:
                    best_loss = loss
                    best_epoch = epoch

                    save_checkpoint(model, f'efficientdet-d{opt.compound_coef}_{epoch}_{step}.pth')

                model.train()

                # Early stopping
                if epoch - best_epoch > opt.es_patience > 0:
                    print('[Info] Stop training at epoch {}. The lowest loss achieved is {}'.format(epoch, best_loss))
                    break
    except KeyboardInterrupt:
        save_checkpoint(model, f'efficientdet-d{opt.compound_coef}_{epoch}_{step}.pth')
        writer.close()
    writer.close()
                           
    if isinstance(model, CustomDataParallel):
        torch.save(model.module.model.state_dict(), os.path.join(opt.model_dir, 'model.pth'))
    else:
        torch.save(model.model.state_dict(), os.path.join(opt.model_dir, 'model.pth'))


def save_checkpoint(model, name):
    if isinstance(model, CustomDataParallel):
        torch.save(model.module.model.state_dict(), os.path.join(opt.saved_path, name))
    else:
        torch.save(model.model.state_dict(), os.path.join(opt.saved_path, name))


if __name__ == '__main__':
    opt = get_args()
    train(opt)


次に、学習スクリプトが読むこむ設定ファイルを作成します。こちらも EfficientDet のリポジトリのものをベースにカスタマイズしています。一番下の行の `obj_list` には、分類したいクラス名をリストとしてセットします。各パラメタ名の後のコロンの後にスペースが必要です。<br>たとえば、`obj_list:<半角スペース>['class1', 'class2']` とします。

In [ ]:
labels

上記セルを実行した出力 `['class1', 'class2'...]` を以下のセルの `obj_list` にコピー＆ペーストします。

In [ ]:
%%writefile src/settings.yml

# project_name: coco_adaptor  # also the folder name of the dataset that under data_path folder
train_set: train
val_set: validation

# mean and std in RGB order, actually this part should remain unchanged as long as your dataset is similar to coco.
mean: [0.485, 0.456, 0.406]
std: [0.229, 0.224, 0.225]

# this is coco anchors, change it if necessary
anchors_scales: '[2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)]'
anchors_ratios: '[(1.0, 1.0), (1.4, 0.7), (0.7, 1.4)]'

# must match your dataset's category_id.
# category_id is one_indexed
obj_list: ['front_face', 'other_face']

次に、学習ジョブで使用するコンテナイメージを作るための Dockerfile を作成します。SageMaker が提供している PyTorch 1.4.0 のコンテナイメージをベースに、必要あライブラリのインストールと、転移学習に必要な学習済みモデルのダウンロードを行っています。このノートブックでは、EfficientDet の coefficient 0 のモデルを使用するため、`efficientdet-d0.pth` をダウンロードします。

**以下のセルの FROM の行の 'us-east-1' の部分をお使いのリージョンに合わせて変更してください。（東京リージョンの場合 `ap-northeast-1`）**

In [ ]:
%%writefile docker/train/Dockerfile
FROM  763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.4.0-gpu-py36-cu101-ubuntu16.04

WORKDIR /opt/app

RUN pip3 install pycocotools numpy opencv-python tqdm tensorboard tensorboardX pyyaml webcolors torchvision==0.5.0
RUN mkdir -p /data/models

RUN wget -O /data/models/efficientdet-d0.pth https://github.com/zylo117/Yet-Another-Efficient-Pytorch/releases/download/1.0/efficientdet-d0.pth
# RUN wget -O /data/models/efficientdet-d2.pth https://github.com/zylo117/Yet-Another-Efficient-Pytorch/releases/download/1.0/efficientdet-d2.pth

WORKDIR /

ここからは、Amazon ECR に関するセットアップをしています。

In [ ]:
ecr_repository = 'efficientdet-pytorch-train'
tag = ':latest'
uri_suffix = 'amazonaws.com'
train_repository_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository + tag)

ベースイメージは Amazon SageMaker が用意している Amazon ECR リポジトリに保存されているため、そこへのアクセス権が必要です。以下のコマンドを実行します。

In [ ]:
!$(aws ecr get-login --region $region --registry-ids 763104351884 --no-include-email)

それでは、以下のセルを実行してコンテナイメージをビルドして ECR にプッシュしましょう。t2.medium のノートブックインスタンスの場合、この処理には 10分程度かかります。

In [ ]:
# Create ECR repository and push docker image
!docker build -t $ecr_repository docker/train
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
!aws ecr create-repository --repository-name $ecr_repository
!docker tag {ecr_repository + tag} $train_repository_uri
!docker push $train_repository_uri

ここまでで、SageMaker の学習ジョブを起動するために必要なコンテナイメージと学習スクリプトの準備が完了しました。それでは学習ジョブを実行しましょう。PyTorch の estimator を作成する際に引数で hyperparameters を設定しています。ここで EfficientDet のハイパーパラメタや転移学習のベースとなる学習済みモデルのパスを指定しています。学習済みモデルのパスは前の手順で Dockerfile で指定したパスと対応しています。`metric_definitions` を設定することで、学習ログから各種メトリクスの数値を抽出し、学習ジョブと紐付けを行います。これらのメトリクスは AWS コンソールの学習ジョブの詳細画面で確認することができます。

以下のセルの `local_mode = False` を `local_mode = True` にすると、ノートブックインスタンス上で学習ジョブを動かすことができます。これをローカルモードを言います。新しいアルゴリズムを試す場合などは、ハイパーパラメタの `num_epochs` を 1 などの小さい値にしてローカルモードで学習ジョブを動かすことで素早くデバッグを行うことができます。ノートブックインスタンスのインスタンスタイプをモデルの学習に十分なスペックのものにして大きなエポック数でローカルモードで学習ジョブを動かすことも可能です。

estimator.fit() で学習ジョブが開始します。引数で学習データ、検証データ、教師データが格納されている Amazon S3 パスを指定しています。

In [ ]:
from sagemaker.estimator import Estimator
from sagemaker.pytorch.estimator import PyTorch
import uuid
import os
import datetime

from sagemaker.local import LocalSession

local_mode = False

if local_mode:
    ss = LocalSession()
    instance_type = 'local_gpu' # or 'local'
else:
    ss = session
    instance_type = 'ml.p3.8xlarge'

# Spot training をする場合は、チェックポイントの設定を推奨
checkpoint_suffix = str(uuid.uuid4())[:8]
checkpoint_s3_path = 's3://{}/checkpoint-{}'.format(bucket, checkpoint_suffix)
checkpoint_local_path='/opt/ml/checkpoints'

job_name = ecr_repository + datetime.datetime.now().strftime("-%Y%m%d-%H%M%S")

pytorch_estimator = PyTorch(
                        entry_point='train.py',
                        source_dir='src',
                        image_uri=train_repository_uri,
                        job_name_base=job_name,
                        role=role, 
                        instance_count=1,
                        sagemaker_session=ss,
                        instance_type=instance_type,
#                         max_run = 5000,
#                         use_spot_instances = 'True',
#                         max_wait = 10000,
                        checkpoint_s3_uri=checkpoint_s3_path,
                        checkpoint_local_path=checkpoint_local_path,
                        output_path="s3://{}/{}/output".format(bucket, job_name),
                        hyperparameters = {'num_epochs': 100, 'batch_size':32, 'head_only':True, 'lr':0.01, 'compound_coef': 0,
                                                       'load_weights': '/data/models/efficientdet-d0.pth'},
                        enable_sagemaker_metrics=True,
#                         profiler_config=profiler_config,
                        metric_definitions = [dict(
                                                                Name = 'Val Cls Loss',
                                                                Regex = 'Classification loss: (.*?),'
                                                            ),
                                                              dict(
                                                                Name = 'Val Reg Loss',
                                                                Regex = 'Regression loss: (.*?),'
                                                            ),
                                                              dict(
                                                                Name = 'Val Total Loss',
                                                                Regex = 'Val Total loss: (.*?)$'
                                                            ),
                                                              dict(
                                                                Name = 'Train Cls Loss',
                                                                Regex = 'Train Cls loss: (.*?),'
                                                            ),
                                                              dict(
                                                                Name = 'Train Reg Loss',
                                                                Regex = 'Train Reg loss: (.*?),'
                                                            ),
                                                              dict(
                                                                Name = 'Train Total Loss',
                                                                Regex = 'Train Total loss: (.*?)$'
                                                            )
                                             ]
                          
)

pytorch_estimator.fit({'train': train_data_s3_path+'/annotations/train/', 
                       'validation': train_data_s3_path+'/annotations/validation/', 
                       'annotations': train_data_s3_path+'/annotations/'}, wait=False)


## ノートブックインスタンス上で推論

先ほど学習したモデルを使って、ノートブックインスタンス上で推論を試してみます。この方法では、モデルをうまく学習できたかどうかをクイックに確認することができます。

まずは、S3 に保存されている学習済みモデルをノートブック インスタンスにダウンロードして解凍します。

In [ ]:
model_path_s3 = pytorch_estimator.model_data

In [ ]:
!aws s3 cp $model_path_s3 ./
!tar zvxf model.tar.gz

以下のセルを実行して、ノートブックインスタンス上で推論を行うスクリプト `efficientdet_pred.py` を作成します。

In [ ]:
%%writefile ./efficientdet_pred.py

# Author: Zylo117

"""
Simple Inference Script of EfficientDet-Pytorch
"""
import sys
sys.path.append('./src')
import time
import torch
from torch.backends import cudnn
from matplotlib import colors
import matplotlib.pyplot as plt
import argparse
import os
import glob
import pickle
import re

from backbone import EfficientDetBackbone
import cv2
import numpy as np
import pandas as pd

from efficientdet.utils import BBoxTransform, ClipBoxes
from utils.utils import preprocess, invert_affine, postprocess, STANDARD_COLORS, standard_to_bgr, get_index_label, plot_one_box


result = []

def display(preds, imgs, cnt, fname):
    for i in range(len(imgs)):
        if len(preds[i]['rois']) == 0:
            continue

        items = []
        for j in range(len(preds[i]['rois'])):
            x1, y1, x2, y2 = preds[i]['rois'][j].astype(np.int)
            obj = obj_list[preds[i]['class_ids'][j]]
            score = float(preds[i]['scores'][j])
            items.append([obj, score])
            plot_one_box(imgs[i], [x1, y1, x2, y2], label=obj,score=score,color=color_list[get_index_label(obj, obj_list)])

        ofilename = os.path.join(args.output_path, fname)
            
        print(items)
        print('result image is saved to ' + ofilename)
        cv2.imwrite(ofilename, imgs[i])

        
parser = argparse.ArgumentParser('EfficientDet inference options')
parser.add_argument('-c', '--compound-coef', type=int, default=0, help='0-8')
parser.add_argument('-im', '--image-path', type=str, default='', help='folder name that has images for inference')
parser.add_argument('-lb', '--label-name', type=str, default='', help='annotation data file name')
parser.add_argument('-cl', '--class-labels', type=str, nargs="*", default='', help='list of class labels')
parser.add_argument('-t', '--conf-thresh', type=float, default=0.2, help='threshold for confidence value')
# parser.add_argument('-w', '--weight-path', type=str, default='', help='weight file path')
parser.add_argument('-o', '--output-path', type=str, default='result', help='output path')
# parser.add_argument('-g', '--use-cuda', type=bool, default=False, help='whether use CUDA')
args = parser.parse_args()

if len(args.class_labels) == 0:
    print('[ERROR] --class-labels is required.')
    sys.exit()
else:
    obj_list = args.class_labels
    obj_list = list(map(lambda x:re.sub('[\[\],]','', x), obj_list))
    

os.makedirs(args.output_path, exist_ok=True)
    
compound_coef = args.compound_coef
force_input_size = None  # set None to use default size

if args.image_path[-3:] == 'jpg':
    img_path_list = [args.image_path]
else:
    img_path_list = glob.glob(args.image_path+'/**/*', recursive=True)

# print(img_path_list)

# replace this part with your project's anchor config
anchor_ratios = [(1.0, 1.0), (1.4, 0.7), (0.7, 1.4)]
anchor_scales = [2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)]

threshold = args.conf_thresh
iou_threshold = 0.2

use_cuda = False
use_float16 = False
cudnn.fastest = False
cudnn.benchmark = False

if use_cuda:
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
else:
    device = 'cpu'

model = EfficientDetBackbone(compound_coef=compound_coef, num_classes=len(obj_list),
                             ratios=anchor_ratios, scales=anchor_scales)
model.load_state_dict(torch.load('model.pth', map_location=torch.device(device) ))
model.requires_grad_(False)
model.eval()

if use_float16:
    model = model.half()


color_list = standard_to_bgr(STANDARD_COLORS)
# tf bilinear interpolation is different from any other's, just make do
input_sizes = [512, 640, 768, 896, 1024, 1280, 1280, 1536]
input_size = input_sizes[compound_coef] if force_input_size is None else force_input_size

for i, ip in enumerate(img_path_list):
    print('-------------------------------')
    
    start = time.time()
    
    img_path = tuple([ip])

    ori_imgs, framed_imgs, framed_metas = preprocess(*img_path, max_size=input_size)

    if use_cuda:
        x = torch.stack([torch.from_numpy(fi).cuda() for fi in framed_imgs], 0)
    else:
        x = torch.stack([torch.from_numpy(fi) for fi in framed_imgs], 0)


    x = x.to(torch.float32 if not use_float16 else torch.float16).permute(0, 3, 1, 2)


    with torch.no_grad():
        features, regression, classification, anchors = model(x)

        regressBoxes = BBoxTransform()
        clipBoxes = ClipBoxes()

        out = postprocess(x,
                          anchors, regression, classification,
                          regressBoxes, clipBoxes,
                          threshold, iou_threshold)
        
        
    out = invert_affine(framed_metas, out)

    print(str(time.time()-start), ' sec.')
    if len(args.label_name) > 0:
        pass
    else:
        display(out, ori_imgs, i, os.path.basename(ip))


result_df = pd.DataFrame(result, columns=['filename', 'ground truth', 'prediction', 'Top', 'Includes'])
result_df.to_csv('result.csv')


作成した `efficientdet_pred.py` を実行して推論を行います。`-im` オプションには入力となる画像が保存されたフォルダのパスを指定します。このサンプルでは、モデルの検証用画像を指定しています。`-o` オプションで指定したフォルダに結果の画像が保存されます。`-cl` オプションには、分類クラスのリストを指定します。モデル学習時と同じ順番で指定してください。

In [ ]:
!python efficientdet_pred.py -t 0.3 -im $coco_output_dir"/annotations/validation/"  -o "result-gt" -cl $labels

上記セルの実行ログに表示されたいずれかの出力ファイル名を以下のセルにセットして実行すると、画像が表示されます。

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

plt.figure(figsize=(7,7))
plt.imshow(mpimg.imread("result-gt/img_00609.jpg"))

## AWS Lambda で推論

モデルがうまく学習できていることが確認できたら、モデルを Lambda にデプロイして推論してみましょう。EfficientDet に必要なライブラリや学習したモデルを含むコンテナイメージを作成し、それを Lambda にデプロイします。

### コンテナイメージとソースコードの準備

ここからは、コンテナイメージを作成するための準備をします。まずは EfficientDet の推論に必要なソースコードと学習済みモデルをコンテナイメージ用にコピーします。

In [ ]:
!mkdir -p ./docker/lambda-inference/app
!cp -r ./src/* ./docker/lambda-inference/app
!cp model.pth ./docker/lambda-inference

次に、Dockerfile を作成します。ノートブックインスタンスでデバッグする際に使用するサンプル画像もイメージの中に埋め込んでいますが、Lambda で実行する際は不要です。

In [ ]:
%%writefile ./docker/lambda-inference/Dockerfile

# Define custom function directory
ARG FUNCTION_DIR="/function"

FROM python:3.6.10-slim-buster
# FROM public.ecr.aws/lambda/python:3.8
    
# Include global arg in this stage of the build
ARG FUNCTION_DIR

# Install aws-lambda-cpp build dependencies
RUN apt-get update && \
  apt-get install -y \
  g++ \
  make \
  cmake \
  unzip \
  libcurl4-openssl-dev \
  libopencv-dev
#   libgl1-mesa-dev
  
RUN pip install \
    torch==1.4.0+cpu torchvision==0.5.0+cpu \
    opencv-python webcolors boto3 \
    -f https://download.pytorch.org/whl/torch_stable.html \
  && rm -rf /root/.cache/pip


# Copy function code
RUN mkdir -p ${FUNCTION_DIR}
COPY app/ ${FUNCTION_DIR}/

COPY model.pth ${FUNCTION_DIR}

# Install the function's dependencies
RUN pip install \
    --target ${FUNCTION_DIR} \
        awslambdaric


# Set working directory to function root directory
WORKDIR ${FUNCTION_DIR}


ENTRYPOINT [ "/usr/local/bin/python", "-m", "awslambdaric" ]
CMD [ "app.handler" ]

次のセルを実行して、Lambda が実行するスクリプト `app.py` を作成します。`class_num` に、モデルを学習した際のクラス数を設定してください。結果の画像にバウンディングボックスが表示されすぎる場合は、`threshold = 0.2` の数値を大きくしてください。

In [ ]:
%%writefile ./docker/lambda-inference/app/app.py

import sys
import time
import torch
from torch.backends import cudnn
import argparse
import os
import glob
import pickle

from backbone import EfficientDetBackbone
import cv2
import numpy as np
import json

from efficientdet.utils import BBoxTransform, ClipBoxes
from utils.utils import preprocess, invert_affine, postprocess, STANDARD_COLORS, standard_to_bgr, get_index_label, plot_one_box

import boto3

version = '1.0'
modelname = 'model.pth'
s3 = boto3.resource('s3')
device = 'cpu'

# replace this part with your project's anchor config
anchor_ratios = [(1.0, 1.0), (1.4, 0.7), (0.7, 1.4)]
anchor_scales = [2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)]
compound_coef = 0
class_num = 2
model = EfficientDetBackbone(compound_coef=compound_coef, num_classes=class_num,
                             ratios=anchor_ratios, scales=anchor_scales)
model.load_state_dict(torch.load(modelname, map_location=torch.device(device) ))
model.requires_grad_(False)
model.eval()

class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)

# def from_s3_to_tmp(event):
def from_s3_to_tmp(bucket, path):
#     bucket = event['bucket']
#     path = event['s3_path']
    
    if 'png' in path:
        filename = '/tmp/input.png'
    elif 'jpg' in path or 'jpeg' in path:
        filename =  '/tmp/input.jpg'
    elif 'pth' in path:
        filename = '/tmp/model.pth'
    else:
        print('[ERROR] wrong file type!')
        return ''

    bucket = s3.Bucket(bucket)
    bucket.download_file(path, filename)
    
    return filename

def gen_result(preds):
    if len(preds[0]['rois']) == 0:
        return {'bbox':[]}

    bbox = []
    for j in range(len(preds[0]['rois'])):
        x1, y1, x2, y2 = preds[0]['rois'][j].astype(np.int)
        cls = preds[0]['class_ids'][j]
        score = float(preds[0]['scores'][j])
        bbox.append({'x1':x1, 'y1':y1, 'x2':x2, 'y2':y2, 'score':score, 'classid':cls})

    return bbox

def handler(event, context):
    
    
    threshold = 0.2
    
    global version
    global modelname
    global model
    global class_num
    global device

    if 's3_path' not in event or 'bucket' not in event:
        return {'result': False}
    
    filename = from_s3_to_tmp(event['bucket'], event['s3_path'])
    
    if filename == '':
        return {'result': False}
    
    
    if 'model_bucket' in event and 'model_s3_path' in event:
        if 'class_num' in event:
            class_num = int(event['class_num'])
        if 'version' in event:
            if version != event['version']:
                print('[INFO] update model: ', event['model_bucket'], event['model_s3_path'])
                modelname = from_s3_to_tmp(event['model_bucket'], event['model_s3_path'])
                version = event['version']
                model = EfficientDetBackbone(compound_coef=compound_coef, num_classes=class_num,
                             ratios=anchor_ratios, scales=anchor_scales)
                model.load_state_dict(torch.load(modelname, map_location=torch.device(device) ))
                model.requires_grad_(False)
                model.eval()
            
    if 'threshold' in event:
        threshold = float(event['threshold'])
    
    
    force_input_size = None  # set None to use default size

    img_path = tuple([filename])

    iou_threshold = 0.2

    use_cuda = False
    use_float16 = False
    cudnn.fastest = False
    cudnn.benchmark = False

    color_list = standard_to_bgr(STANDARD_COLORS)
    # tf bilinear interpolation is different from any other's, just make do
    input_sizes = [512, 640, 768, 896, 1024, 1280, 1280, 1536]
    input_size = input_sizes[compound_coef] if force_input_size is None else force_input_size
    ori_imgs, framed_imgs, framed_metas = preprocess(*img_path, max_size=input_size)

    x = torch.stack([torch.from_numpy(fi) for fi in framed_imgs], 0)
    x = x.to(torch.float32 if not use_float16 else torch.float16).permute(0, 3, 1, 2)

#     pt1 = time.time()
#     model = EfficientDetBackbone(compound_coef=compound_coef, num_classes=class_num,
#                                  ratios=anchor_ratios, scales=anchor_scales)
#     model.load_state_dict(torch.load(modelname, map_location=torch.device(device) ))
#     pt2 = time.time()
#     model.requires_grad_(False)
#     model.eval()

    if use_float16:
        model = model.half()

    with torch.no_grad():
        pt3 = time.time()
        features, regression, classification, anchors = model(x)
        pt4 = time.time()
        
        regressBoxes = BBoxTransform()
        clipBoxes = ClipBoxes()

        out = postprocess(x,
                      anchors, regression, classification,
                      regressBoxes, clipBoxes,
                      threshold, iou_threshold)
        
    out = invert_affine(framed_metas, out)
    out = gen_result(out)
    
    result = json.dumps(out, cls=MyEncoder)
    pt5 = time.time()

    print(result)
#     print('pt2-pt1 (load model): ', str(pt2-pt1))
    print('pt4-pt3 (inference): ', str(pt4-pt3))
    print('pt5-pt4 (post process): ', str(pt5-pt4))
    
    return {
        'statusCode'        : 200,
        'result':result
    }

## AWS Lambda で推論

ここからは、AWS Lambda で推論を実行するための手順を行なってきます。

### コンテナイメージのビルドと Amazon ECR へのプッシュ

コンテナやソースコードの動作確認ができたら、次はそれらを Lambda にデプロイします。以下のセルを実行して、Amazon ECR のリポジトリ名などを指定します。

In [ ]:
ecr_repository_lambda_inference = 'efficientdet-lambda-inference'
tag = ':latest'
uri_suffix = 'amazonaws.com'
inference_repository_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository_lambda_inference + tag)

以下のセルを実行して、コンテナイメージをビルドして ECR にプッシュします。ノートブックインスタンスで Lambda 用コンテナの動作確認をした際にイメージのビルドは完了しているので、この処理はイメージを　ECR　にプッシュするための 1分ほどで完了します。

In [ ]:
# Create ECR repository and push docker image
!docker build -t $ecr_repository_lambda_inference docker/lambda-inference
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
!aws ecr create-repository --repository-name $ecr_repository_lambda_inference
!docker tag {ecr_repository_lambda_inference + tag} $inference_repository_uri
!docker push $inference_repository_uri

### コンテナイメージを AWS Lambda にデプロイ

ここからは、ノートブックインスタンスを離れて、AWS Lambda のコンソール操作になります。AWS コンソールから AWS Lambda のコンソールにアクセスしてください。その後、以下の手順を実施して先ほど作成したコンテナイメージが動作する Lambda 関数を作成してください。

1. AWS Lambda コンソールで、「関数の作成」をクリック
1. 「以下のいずれかのオプションを選択して、関数を作成します。」と書かれた部分で「コンテナイメージ」を選択
1. 「関数名」に任意の関数名を入力
1. 「イメージを参照」ボタンをクリックして先ほど作成した Lambda 用のコンテナイメージ（efficientdet-lambda-inference）を選択<br>
「Amazon ECR イメージリポジトリ」のプルダウンメニューに作成したはずのリポジトリがなかったり、コンテナイメージがない場合は一つ上のセルの実行時に何らかのエラーが出ている可能性があるので確認してください。**モデルの学習の際に使用したコンテナを選ばないようにご注意ください**
1. 「関数の作成」をクリック
1. 作成した関数名をクリックしてから「設定」タブをクリック
1. 左側のメニューで「一般設定」を選択し、「編集」ボタンをクリック
1. 「メモリ」を512 MB に変更、「タイムアウト」を 1分 に変更して「保存」ボタンをクリック
1. 左側のメニューで「アクセス権限」をクリックして、「ロール名」のリンクをクリック（IAM のコンソールが別タブで開く）
1. 表示されたロールに「AmazonS3FullAccess」ポリシーをアタッチ

一度関数を作成したあとでコンテナイメージを更新したい場合は、作成した関数のコンソール画面で「イメージ」タブを選択し、「新しいイメージをデプロイ」をクリックして更新したいコンテナイメージを選択してください。

### 作成した Lambda 関数を実行

推論したい画像を S3 にアップロードしてから以下のセルを実行して、Lambda を使った推論を行います。以下のセルの 'bucket_name', 'image_path', lambda_function_name' をご自身の環境に合わせて書き換えてください。Lambda のメモリが不足することがあるので推論の入力として指定する画像サイズは短辺が 1000画素程度になるようリサイズしてください。

実行結果のログに`{'statusCode': 200, 'result': '[{"x1": 366, "y1": 853, "x2": 627, "y2": 974, "score": 0.2866290807723999, "classid": 0}]'}` のようなテキストが表示されていれば OK です。推論結果を反映させた画像が、`img_inferred_dn.jpg` ようなファイル名で保存されます。

#### モデルの更新
推論に使用するモデルを更新したい場合は、あらかじめ更新したいモデルファイル（model.pth） を S3 にアップロードし、Lambda 関数を Invoke する際に、引数で version（モデルのバージョン）、model_bucket（モデルを保存している S3 バケット名）、model_s3_path（モデルを保存しているバケット名以下のパス）を指定すると、指定されたモデルを使った推論結果を取得できるようになります。不要なダウンロードを回避するために、関数が記憶している version と異なる version が指定された場合のみ指定されたモデルを S3 からダウンロードするようにしているので、モデルを更新したい場合は必ず適切な version を指定してください。version、model_bucket、model_s3_path を指定しなくても関数を実行することは可能ですが、更新されたモデルを使って推論されるかデフォルトのモデルを使って推論されるかはタイミングに依存します（コールドスタートになった場合はデフォルトのモデルが使用されます）のでご注意ください。

In [ ]:
# %%writefile ./access.py

import sys
sys.path.append('./src')
import json
import boto3
import time
import cv2
from utils.utils import plot_one_box, standard_to_bgr, STANDARD_COLORS, get_index_label

#--------------------------------
# 推論したい画像が格納された S3 バケット名
bucket_name = 'bucket'

# 推論したい画像が格納された S3 バケット名以下のパス
image_path = 'dir/xxx.jpg'

# Lambda 関数名
lambda_function_name = 'efficientdet-inference'
#--------------------------------
# 分類クラスのリスト（今まで同じ順番で）
obj_list = labels
# coefficient（今までと同じ値）
compound_coef = 0
#--------------------------------


def save_image(bboxes, img):
    color_list = standard_to_bgr(STANDARD_COLORS)
    
    for b in bboxes:
        x1= b['x1']
        y1= b['y1']
        x2= b['x2']
        y2= b['y2']
        obj = obj_list[b['classid']]
        score = b['score']
        plot_one_box(img, [x1, y1, x2, y2], label=obj,score=score,color=color_list[get_index_label(obj, obj_list)])

    ofilename = 'img_inferred_d'+str(compound_coef)+'.jpg'
    print('Image was saved as ', ofilename)
    cv2.imwrite(ofilename, img)

start = time.time()
l = boto3.Session().client('lambda').invoke(
    FunctionName=lambda_function_name,
    InvocationType='RequestResponse', # Event or RequestResponse
#     Payload=json.dumps({'bucket': bucket_name, 's3_path': image_path, 'model_bucket': bucket_name, 'version': '1.1', 'model_s3_path': 'data/efficientdet/awscats/model.pth'})
    Payload=json.dumps({'bucket': bucket_name, 's3_path': image_path})
)
j = json.loads(l['Payload'].read())
print('Time for inference: ', round(time.time()-start, 2), ' sec.')

print(j)
bbox = json.loads(j['result'])

tmp_img = os.path.basename(image_path)
!aws s3 cp 's3://'$bucket_name/$image_path ./

img = cv2.imread(tmp_img)
save_image(bbox, img)


以下のセルを実行して、出力された画像を表示します。

In [ ]:
plt.figure(figsize=(10,7))
plt.imshow(mpimg.imread("img_inferred_d0.jpg"))

## リソースの削除

このノートブックを実行するのに使用したノートブックインスタンスや、ノートブックで作成したものを削除して不要な課金を回避してください。このノートブックで作成したものは以下のとおりです。今後使用しないことを確認の上削除してください。

1. `sagemaker-<region>-<account ID>` という名前の S3 バケット
1. 学習データをアップロードした S3 バケット
1. Amazon ECR のリポジトリとイメージ
1. AWS Lambda 関数

## [Option] Lambdaにデプロイする前にローカルで動作確認

モデルを Lambda にデプロイする前にローカルで動作確認をしておきたい場合、こちらのツールキットのリポジトリをクローンします。

In [ ]:
!git clone https://github.com/aws/aws-lambda-python-runtime-interface-client.git

### ノートブックインスタンス上でコンテナとソースコードの動作確認

以下のコマンドを実行して、ノートブックインスタンス上で関数のデバッグに使用するツールをインストールします。

In [ ]:
!mkdir -p ~/.aws-lambda-rie && \
    curl -Lo ~/.aws-lambda-rie/aws-lambda-rie https://github.com/aws/aws-lambda-runtime-interface-emulator/releases/latest/download/aws-lambda-rie && \
    chmod +x ~/.aws-lambda-rie/aws-lambda-rie

以下のセルを実行して、Lambda 推論用のコンテナイメージをビルドして実行するスクリプトを作成します。コマンドで指定している各種パスはご自身の環境に合わせて書き換えてください。以下のセルはこのサンプル用の記述になっています。

In [ ]:
%%writefile ./build_and_run.sh

docker build -t efficientdet-lambda-inference docker/lambda-inference
docker run --name lambda --rm -p 9000:8080 \
          -v ~/.aws-lambda-rie:/aws-lambda \
            --entrypoint /aws-lambda/aws-lambda-rie \
             efficientdet-lambda-inference \
                /usr/local/bin/python -m awslambdaric app.handler

Terminal で、作成したスクリプトを実行します。
**以下のセルを実行する前に以下の手順を実施してください。**

1. Jupyter のファイルブラウザのブラウザタブ（このノートブックのタブの左側にあることが多い）を表示し、右上にある New -> Terminal をクリック
1. Terminal が表示されたら以下のコマンドを実行<br>
```
$ cd SageMaker
$ sh build_and_run.sh
```

すると、コンテナイメージが実行されます。ログの表示が停止したら推論リクエストの待機状態なので、以下のセルを実行します。`{"statusCode": 200, "result": "[{\"x1\": 905, \"y1\": 1458, \"x2\": 2004, \"y2\": 1845, \"score\": 0.5928193926811218, \"classid\": 0}]"}` のようなログが表示されたら成功です。

In [ ]:
!curl -XPOST "http://localhost:9000/2015-03-31/functions/function/invocations" -d '{"bucket": "bucket_name", "s3_path": "path/frame_0003.jpeg"}'